In [0]:
## Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from ast import literal_eval

In [0]:
class LSH :
  
    # Random limits, const
    __max_rand =  1000
    __min_rand = -1000
    
    # Constructor
    def __init__(self, k, L, embedding_size = 150):
        
        # Number of hash function
        self.k = k
        
        # Number of attempts
        self.L = L
        
        # Embedding length 
        self.embedding_size = embedding_size
        
        # Random matrices
        self.normalized_random_matrices = []
        
        for i in range(self.L):
            random_matrix = np.random.randint(self.__min_rand, self.__max_rand,(self.k, self.embedding_size));
            
            # Append normalized random matrices
            self.normalized_random_matrices.append(normalize(random_matrix, axis=1, norm='l1'))
        
    
    # Locality Sensitive hash function
    def locality_sensitive_hash(self, embedding, matrix_index):
        out = 0
      
        for h in self.normalized_random_matrices[matrix_index]:
            if (np.dot(h, embedding) >= 0):
                out = (out << 1) | 1
            else:
                out = (out << 1) | 0

        return out
      
    # Divide in buckets using L-th matrix
    def divide_in_buckets(self, embeddings, matrix_index):
        out = {}
        for embedding in embeddings:
            hash = self.locality_sensitive_hash(embedding, matrix_index)
            if (hash in out):
                out[hash].append(embedding)
            else:
                out[hash] = [embedding]
            
        return out  


In [0]:
test = LSH(k=5, L=2, embedding_size=10)
print(test.normalized_random_matrices)

print(test.locality_sensitive_hash([1,2,3,4,5,6,7,8,9,10], 1))

In [0]:
## almost equals
embeddings = [[1.2345,2,3,4,5,6,10.4,8,9,10],[1,2,3,4,5,6,7,8,9,10],[1,2,3,5,5,6,7,8,9,10]]
print(len(test.divide_in_buckets(embeddings, 1)))

## not equals
embeddings = np.random.randint(-10000, 10000,(10000, 10))
print(len(test.divide_in_buckets(embeddings, 1)))


In [0]:
## TEST BLOCKING PERFORMANCE
#    Basta che per ogni tupla vado a prendere la sua corrispondente, ne calcolo
#     i vari L hash e controllo che almeno uno sia uguale e incremento un
#     contatore. La precisione è contatore/numero di tuple controllate, giusto?
def performance_test(filtered_dataset, k, L, embedding_size):
    
    match_found = 0
    
    lsh = LSH(k, L, embedding_size)
    
    # for each elemt in dataset
    for index, row in filtered_dataset.iterrows():
        x_embedding = np.array(literal_eval(row['left_table']))
        y_embedding = np.array(literal_eval(row['right_table']))
          
        x_hashs = set()
        y_hashs = set()
        for i in range(L):
            x_hashs.add(lsh.locality_sensitive_hash(x_embedding, i))
            y_hashs.add(lsh.locality_sensitive_hash(y_embedding, i))
        
        if (len(set.intersection(x_hashs, y_hashs)) > 0):
            match_found += 1
  
    
    return match_found / len(filtered_dataset.index)

In [44]:
## TEST

## mount google drive fs
from google.colab import drive
drive.mount('/content/gdrive')

## Open dataset 
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/lsh-testing-data.csv")

## Remove 0 labled
df = df[df.label == 1]

precision_max = 0
k_max = 0
L_max = 0
for k in range(30):
    for L in range(10):
        precision = performance_test(df, k + 1, L + 1, 50)
        print("K: {0}, L: {1}, Precision:{2}".format(k + 1, L + 1, precision))
        if (precision >= precision_max):
            precision_max = precision
            k_max = k + 1
            L_max = L + 1

print("Max precision: {0}, k: {1}, L: {2}".format(precision_max, k_max, L_max))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
K: 1, L: 1, Precision:0.8181818181818182
K: 1, L: 2, Precision:0.6212121212121212
K: 1, L: 3, Precision:1.0
K: 1, L: 4, Precision:1.0
K: 1, L: 5, Precision:1.0
K: 1, L: 6, Precision:1.0
K: 1, L: 7, Precision:1.0
K: 1, L: 8, Precision:1.0
K: 1, L: 9, Precision:1.0
K: 1, L: 10, Precision:1.0
K: 2, L: 1, Precision:0.24242424242424243
K: 2, L: 2, Precision:0.5909090909090909
K: 2, L: 3, Precision:0.8484848484848485
K: 2, L: 4, Precision:1.0
K: 2, L: 5, Precision:1.0
K: 2, L: 6, Precision:1.0
K: 2, L: 7, Precision:1.0
K: 2, L: 8, Precision:1.0
K: 2, L: 9, Precision:1.0
K: 2, L: 10, Precision:1.0
K: 3, L: 1, Precision:0.045454545454545456
K: 3, L: 2, Precision:0.42424242424242425
K: 3, L: 3, Precision:0.7575757575757576
K: 3, L: 4, Precision:0.7121212121212122
K: 3, L: 5, Precision:0.9848484848484849
K: 3, L: 6, Precision:1.0
K: 3, L: 7, Precision:1.0
K: 3, L: 8,